# **Day 7: Camel Cards**

This problem doesn't seem that hard - I just need to parse the different hands, figure out what they're worth, and then compare them. 

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
from functools import cmp_to_key
import re

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
with open("data/input-files/day-07-input.txt", "r") as txt_file:
    input_data = txt_file.readlines()

# Parsing the Input Data

In [4]:
# Parse the input data into a DataFrame
hands_df_records = []
for line in input_data:
    hand, bid = line.strip().split(" ")
    hands_df_records.append({"hand": hand, "bid": int(bid)})
hands_df = pd.DataFrame(hands_df_records)

Now that I've got a DataFrame of the different hands, I need to order them according to their value. The "type" rules from strongest to weakest are as follows: 

```
Five of a kind, where all five cards have the same label: AAAAA
Four of a kind, where four cards have the same label and one card has a different label: AA8AA
Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
High card, where all cards' labels are distinct: 23456
```

I'll apply all of the types now: 

In [5]:
# Indicate the strengths of the cards
cards_ordered_by_strength = [
    "A",
    "K",
    "Q",
    "J",
    "T",
    "9",
    "8",
    "7",
    "6",
    "5",
    "4",
    "3",
    "2",
]
card_to_strength = {
    card: len(cards_ordered_by_strength) - idx
    for idx, card in enumerate(cards_ordered_by_strength)
}


def determine_hand_type(hand):
    """
    This method will determine a hand's type, and then return it as a string.
    """

    # Parse the hand into strength values
    hand_strength_values = [card_to_strength.get(card) for card in hand]

    # Determine the frequency of each card type
    freq_ct_sorted = sorted(
        [
            hand_strength_values.count(card_val)
            for card_val in set(hand_strength_values)
        ],
        reverse=True,
    )

    # Determine what type of hand this is
    if freq_ct_sorted[0] == 5:
        hand_type = "five_of_a_kind"
    elif freq_ct_sorted[0] == 4:
        hand_type = "four_of_a_kind"
    elif freq_ct_sorted == [3, 2]:
        hand_type = "full_house"
    elif freq_ct_sorted[0] == 3:
        hand_type = "three_of_a_kind"
    elif freq_ct_sorted[:2] == [2, 2]:
        hand_type = "two_pair"
    elif freq_ct_sorted[0] == 2:
        hand_type = "one_pair"
    else:
        hand_type = "high_card"

    # Return the type of the hand
    return hand_type

# Add a column containing the hand type
hands_df["hand_type"] = hands_df["hand"].apply(lambda x: determine_hand_type(x))

# Show off a sample of this DataFrame
hands_df.sample(5)

hand  bid        hand_type
592  7T656  488         one_pair
923  JJJ7A  382  three_of_a_kind
6    977K9  552         two_pair
223  56884   67         one_pair
425  53425  680         one_pair

Now that we know each of the cards' types, we can sort them according to their rank. 

In [6]:
# Create a dictionary mapping the hand types to their strength
hand_types_ordered_by_strength = ["five_of_a_kind", "four_of_a_kind", "full_house", "three_of_a_kind", "two_pair", "one_pair", "high_card"]
hand_type_to_strength = {
    hand_type: len(hand_types_ordered_by_strength) - idx
    for idx, hand_type in enumerate(hand_types_ordered_by_strength)
}

# Add some helpful columns to the `hands_df`, and then sort by hand type strength 
hands_df["hand_type_value"] = hands_df["hand_type"].apply(lambda hand_type: hand_type_to_strength.get(hand_type))
hands_df["card_values_in_order"] = hands_df["hand"].apply(
    lambda hand: [card_to_strength.get(card) for card in hand]
)
hands_df = hands_df.sort_values("hand_type_value", ascending=False)

# Show the first 7 rows of the hands_df
hands_df.head(7)

hand  bid       hand_type  hand_type_value  card_values_in_order
282  JJJJJ  512  five_of_a_kind                7  [10, 10, 10, 10, 10]
819  33373  906  four_of_a_kind                6       [2, 2, 2, 6, 2]
815  7777T  915  four_of_a_kind                6       [6, 6, 6, 6, 9]
286  2222K  432  four_of_a_kind                6      [1, 1, 1, 1, 12]
299  999K9   68  four_of_a_kind                6      [8, 8, 8, 12, 8]
317  66J66  720  four_of_a_kind                6      [5, 5, 10, 5, 5]
320  888J8  580  four_of_a_kind                6      [7, 7, 7, 10, 7]

Next, we need to sort each of the different hands within each of their "hand types". This code ended up being a little messy - I ought to try and learn how to do this within Pandas more efficiently (i.e., using the `key` argument in [the `sort_values` function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html#pandas-dataframe-sort-values)).

In [7]:
def compare_hands(hand_1_strengths, hand_2_strengths):
    """
    This is a custom sorting method that will help me sort the hands according
    to their strength.
    """

    # Iterate through each of the cards in the hand
    for idx in range(len(hand_1_strengths)):
        hand_1_cur_card_strength = hand_1_strengths[idx]
        hand_2_cur_card_strength = hand_2_strengths[idx]

        if hand_1_cur_card_strength == hand_2_cur_card_strength:
            continue

        if hand_2_cur_card_strength > hand_1_cur_card_strength:
            return -1

        if hand_2_cur_card_strength < hand_1_cur_card_strength:
            return 1

    # If we've made it through the cards and they're exactly the same, return 0
    return 0


# Iterate through each of the different hand types and sort the hands
sorted_hands_df_list = []
for hand_type in hand_types_ordered_by_strength:
    cur_type_hands_df = hands_df.query("hand_type==@hand_type").copy()
    if len(cur_type_hands_df) == 0:
        continue
    sorted_hand_value_lists = sorted(
        list(cur_type_hands_df["card_values_in_order"]),
        key=cmp_to_key(compare_hands),
        reverse=True,
    )
    sorting_helper_df = pd.DataFrame({"card_values_in_order": sorted_hand_value_lists})
    sorting_helper_df["rank_within_hand_type"] = list(range(len(sorting_helper_df)))
    sorting_helper_df["df_join_key"] = sorting_helper_df.apply(
        lambda row: "-".join([f"{val}" for val in row.card_values_in_order]), axis=1
    )
    cur_type_hands_df["df_join_key"] = cur_type_hands_df.apply(
        lambda row: "-".join([f"{val}" for val in row.card_values_in_order]), axis=1
    )
    cur_type_hands_df = cur_type_hands_df.merge(
        sorting_helper_df, on="df_join_key"
    ).sort_values("rank_within_hand_type", ascending=True)
    cur_type_hands_df = cur_type_hands_df.drop(
        columns=["df_join_key", "card_values_in_order_y"]
    ).rename(columns={"card_values_in_order_x": "card_values_in_order"})
    sorted_hands_df_list.append(cur_type_hands_df.copy())

# Concatenate all of the DataFrames
sorted_hands_df = pd.concat(sorted_hands_df_list)

# Show the first couple values in this DataFrame
sorted_hands_df.head(7)

hand  bid       hand_type  hand_type_value  card_values_in_order  \
0   JJJJJ  512  five_of_a_kind                7  [10, 10, 10, 10, 10]   
23  AAAA5  764  four_of_a_kind                6   [13, 13, 13, 13, 4]   
89  AAA6A  181  four_of_a_kind                6   [13, 13, 13, 5, 13]   
6   AA9AA  823  four_of_a_kind                6   [13, 13, 8, 13, 13]   
75  AA8AA  895  four_of_a_kind                6   [13, 13, 7, 13, 13]   
30  AA7AA   89  four_of_a_kind                6   [13, 13, 6, 13, 13]   
17  AKAAA  323  four_of_a_kind                6  [13, 12, 13, 13, 13]   

    rank_within_hand_type  
0                       0  
23                      0  
89                      1  
6                       2  
75                      3  
30                      4  
17                      5

Now that I've sorted everything, I can determine the winning values:

In [8]:
# Add an "overall hand rank" column, and then determine the value of the hands
sorted_hands_df["overall_hand_rank"] = [len(sorted_hands_df)-idx for idx in range(len(sorted_hands_df))]
sorted_hands_df["hand_value"] = sorted_hands_df.apply(
    lambda row: row.bid * row.overall_hand_rank,
    axis=1
)

# Print the total sum of the hand values
total_winnings = sorted_hands_df["hand_value"].sum()
print(f"The total winnings is '{total_winnings}'")

The total winnings is '247961593'
